Training the PHSI model on a skewed mass-spring system with damping and external force. The goal is to see how the pruning hyperparameters affect the pruning and the learning of terms with small coefficients.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
#import from phlearn
from phlearn.phnns import PseudoHamiltonianNN
from phlearn.phnns import R_estimator, HamiltonianNN, ExternalForcesNN
from phlearn.phnns import npoints_to_ntrajectories_tsample

#import from phsi
from phsi.phsystems import init_smsdsystem
from phsi.phnns import PHSI, ExternalForcesSI, BaselineSI, generate_dataset, train
ttype = torch.float32
torch.set_default_dtype(ttype)

In [ ]:
#set up parameters
ntrainingpoints = 5000
sampling_time = 1/10
t_max = 10
integrator = 'srk4'
F_timedependent = True
F_statedependent = False
hidden_dim = 100
learning_rate = 5e-3
batch_size = 32
epochs = 150
l1_param_forces = 0.01
l1_param_dissipation = 0.
l1_val_params = 0.1
pruning_interval = 20
pruning_epsilon = 5e-2
shuffle = True
noise_std = 0.
seed = 1
verbose = True
ntrajectories_val = 0

In [ ]:
print('Set up skewed mass-spring system')
ntrajectories_train, t_sample = npoints_to_ntrajectories_tsample(ntrainingpoints, t_max, sampling_time)

gamma = 0.07 # gamma
print('skewed parameter:', gamma)
pH_system = init_smsdsystem(skewed=gamma)
pH_system.seed(seed=seed)
damped_states = [False, True]

nstates = pH_system.nstates

In [ ]:
#generate data
traindata = generate_dataset(pH_system, integrator, ntrajectories_train, t_sample, nsamples=ntrainingpoints, noise_std=noise_std)
valdata = generate_dataset(pH_system, integrator, ntrajectories_val, t_sample, noise_std=noise_std)

#generate noisy data
noise_std = 0.2
print('noise_std:', noise_std)
traindata_noise = generate_dataset(pH_system, integrator, ntrajectories_train, t_sample, nsamples=ntrainingpoints, noise_std=noise_std)
valdata_noise = generate_dataset(pH_system, integrator, ntrajectories_val, t_sample, noise_std=noise_std)    

In [ ]:
#set up PHSI model
function_space_ext = np.array([1,1])
external_forces_filter = np.zeros(nstates)
external_forces_filter[-1] = 1
ext_forces_si = ExternalForcesSI(nstates, F_timedependent, F_statedependent, \
                                function_space_ext, degrees=3)
ext_forces_si_noise = ExternalForcesSI(nstates, F_timedependent, F_statedependent, \
                                function_space_ext, degrees=3)

r_estimator = R_estimator(damped_states)
r_estimator_noise = R_estimator(damped_states)
function_space = np.array([1,0])
model_phsi=PHSI(nstates, pH_system.structure_matrix, function_space, \
                ext_forces_si, r_estimator, degrees=3, full_si=True)
model_phsi_noise=PHSI(nstates, pH_system.structure_matrix, function_space, \
                ext_forces_si_noise, r_estimator_noise, degrees=3, full_si=True)

optimizer_phsi = torch.optim.Adam(model_phsi.parameters(), lr=learning_rate, weight_decay=1e-4)
optimizer_phsi_noise = torch.optim.Adam(model_phsi_noise.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
print('Train PHSI model')
phsi, vloss_phsi = train(model_phsi, integrator, traindata, optimizer_phsi, valdata=valdata, epochs=epochs,
                  batch_size=batch_size, shuffle=shuffle, l1_param_forces=l1_param_forces,
                  l1_param_dissipation=l1_param_dissipation, l1_val_params=l1_val_params, prune_val=pruning_interval, prune_eps=pruning_epsilon,
                  loss_fn=torch.nn.MSELoss(), verbose=verbose, return_best=True)

In [ ]:
print('Train PHSI model on noisy data')
phsi_noise, vloss_phsi = train(model_phsi_noise, integrator, traindata_noise, optimizer_phsi_noise, valdata=valdata, epochs=epochs,
                  batch_size=batch_size, shuffle=shuffle, l1_param_forces=l1_param_forces,
                  l1_param_dissipation=l1_param_dissipation, l1_val_params=l1_val_params, prune_val=pruning_interval, prune_eps=pruning_epsilon,
                  loss_fn=torch.nn.MSELoss(), verbose=verbose, return_best=True)

The exact results depend on the random parameters in the stochastic Adam optimizer. We give here more details on the results reported in our paper, achieved with different choices of `gamma` and `pruning_epsilon`.